In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
train =  pd.read_csv('/content/drive/MyDrive/RAG/train_lenplus.csv', index_col  = 0)
test =  pd.read_csv('/content/drive/MyDrive/RAG/test_one.csv', index_col  = 0)

In [ ]:
train

In [5]:
test

,사고원인
0,"펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측..."
1,작업자의 불안전한 행동(숫돌 측면 사용) 및 보안면 미 착용
2,작업자가 작업을 위해 이동 중 전방을 주시하지 않아 발을 헛디뎌 계단에서 굴러 넘어짐
3,작업 발판 위 벽돌 잔재를 밟고 넘어짐
4,점심식사를 위한 이동시 작업자 부주의로 인한 추락사고 발생
...,...
959,약한 석재가 자연 파단하여 늑골 골절 됨
960,근로자가 작업 중 이동가능한 통로 확보 미흡
961,작업자가 믹서트럭 슈트아래로 불안전하게 이동하면서 콘크리트 통사이에 손가락이 끼임
962,깨진타일을 손으로 주워 마대자루에 넣을려고 하던 중 파편이 박힘


In [6]:
# !huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `rw_only_token` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authent

### 모델 불러오기

In [4]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow

In [5]:
%%capture
import os
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### 데이터셋 가공

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


# tokenizer =  AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B")

In [7]:
!pip install datasets

In [8]:
from datasets import DatasetDict, Dataset

In [9]:
train_dataset = DatasetDict()
test_dataset = DatasetDict()


train_dataset = Dataset.from_pandas(train)
test_dataset['test'] = Dataset.from_pandas(test)

In [14]:
train_dataset.num_rows

6614

In [15]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

instruction = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    global instruction
    instructions = [instruction for _ in range(6614)]
    inputs       = examples["사고원인"]
    outputs      = examples["재발방지대책 및 향후조치계획"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
    pass

# from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset_preprocessed = train_dataset.map(formatting_prompts_func, batched = True,)


Map:   0%|          | 0/6614 [00:00<?, ? examples/s]

### 학습

In [19]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_preprocessed,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/6614 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/6614 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/6614 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/6614 [00:00<?, ? examples/s]

In [20]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,614 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.213900
2,3.189500
3,3.190100
4,3.000900
5,2.967000
6,2.657000
7,2.402900
8,2.215600
9,2.091000
10,1.867100


In [48]:
model.save_pretrained("/content/drive/MyDrive/RAG/First_trained_model")  # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/RAG/First_trained_model")

('/content/drive/MyDrive/RAG/First_trained_model/tokenizer_config.json',
 '/content/drive/MyDrive/RAG/First_trained_model/special_tokens_map.json',
 '/content/drive/MyDrive/RAG/First_trained_model/tokenizer.json')

### 추론

In [10]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/RAG/First_trained_model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
"""

instruction = '당신은 사고원인을 보고 재발방지대책 및 향후 조치계획을 생성해내는 AI입니다. Input에 맞는 알맞은 Response를 생성해 네세요.'

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func_test(examples):
    global instruction
    instructions = [instruction for _ in range(964)]
    inputs       = examples["사고원인"]
    # outputs      = examples["재발방지대책 및 향후조치계획"]
    texts = []
    for instruction, input in zip(instructions, inputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
    pass


test_dataset_preprocessed = test_dataset.map(formatting_prompts_func_test, batched = True,)

Map:   0%|          | 0/964 [00:00<?, ? examples/s]

In [16]:
# del inputs, outputs
torch.cuda.empty_cache()

In [20]:
from tqdm import tqdm
from torch.utils.data import DataLoader

FastLanguageModel.for_inference(model)

# 배치 크기 설정
batch_size = 16  # VRAM 상황에 따라 조절

# DataLoader 생성
dataloader = DataLoader(test_dataset_preprocessed['test'], batch_size=batch_size, shuffle=False)

all_outputs = []

for batch in tqdm(dataloader):
    # 배치 데이터 전처리
    inputs = tokenizer(
        batch['text'],
        padding=True,
        truncation=True,
        return_tensors='pt'
    ).to("cuda")

    # 생성
    outputs = model.generate(**inputs, max_new_tokens=128)

    # CPU로 이동 후 저장
    all_outputs.extend(outputs.cpu().numpy())

    # 메모리 정리
    del inputs, outputs
    torch.cuda.empty_cache()

# 모든 출력 결과 처리
decoded_outputs = tokenizer.batch_decode(all_outputs, skip_special_tokens=True)


100%|██████████| 61/61 [58:32<00:00, 57.58s/it]


In [29]:
outputs = [decoded_output.split('###')[-1].split('Response:\n')[-1] for decoded_output in decoded_outputs]

576

In [32]:
pd.DataFrame(outputs).to_csv("/content/drive/MyDrive/RAG/first_answer.csv")

In [4]:
# !pip install llama_cpp_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama_cpp_python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4601121 sha256=9aa487b2a748ebbe2cbfad77d2cd42865d439e8634dab39901bdc0aa11e58a88
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama_cpp_python


In [7]:
# Save to q4_k_m GGUF
# model.save_pretrained_gguf("/content/drive/MyDrive/Model/llama-3.1-8B(GGUF)", tokenizer, quantization_method = "q4_k_m")


Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.96 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 31%|███▏      | 10/32 [00:00<00:00, 51.03it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:10<00:00,  2.22s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving /content/drive/MyDrive/Model/llama-3.1-8B(GGUF)/pytorch_model-00001-of-00004.bin...
Unsloth: Saving /content/drive/MyDrive/Model/llama-3.1-8B(GGUF)/pytorch_model-00002-of-00004.bin...
Unsloth: Saving /content/drive/MyDrive/Model/llama-3.1-8B(GGUF)/pytorch_model-00003-of-00004.bin...
Unsloth: Saving /content/drive/MyDrive/Model/llama-3.1-8B(GGUF)/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at /content/drive/MyDrive/Model/llama-3.1-8B(GGUF) into f16 GGUF format.
The output location will be /content/drive/MyDrive/Model/llama-3.1-8B(GGUF)/unsloth.F16.gguf
This might take 3 minutes...
/bin/sh: 1: Syntax error: "(" unexpected


RuntimeError: Unsloth: Quantization failed for /content/drive/MyDrive/Model/llama-3.1-8B(GGUF)/unsloth.F16.gguf
You might have to compile llama.cpp yourself, then run this again.
You do not need to close this Python program. Run the following commands in a new terminal:
You must run this in the same folder as you're saving your model.
git clone --recursive https://github.com/ggerganov/llama.cpp
cd llama.cpp && make clean && make all -j
Once that's done, redo the quantization.

In [8]:
# !git clone --recursive https://github.com/ggerganov/llama.cpp
# !cd llama.cpp && make clean && make all -j

fatal: destination path 'llama.cpp' already exists and is not an empty directory.
Makefile:2: *** The Makefile build is deprecated. Use the CMake build instead. For more details, see https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md.  Stop.
